In [ ]:
from PIL import Image
import os
import glob
from pathlib import Path
import json
import os, shutil


def zoom_center(image, zoom_factor):
    # Get image dimensions
    width, height = image.size

    # Calculate the center coordinates
    center_x, center_y = width // 2, height // 2

    # Calculate the size of the zoomed-in region
    zoomed_width = int(width * zoom_factor)
    zoomed_height = int(height * zoom_factor)

    # Calculate the coordinates for cropping
    crop_x1 = max(center_x - zoomed_width // 2, 0)
    crop_x2 = min(center_x + zoomed_width // 2, width)
    crop_y1 = max(center_y - zoomed_height // 2, 0)
    crop_y2 = min(center_y + zoomed_height // 2, height)

    # Crop the region around the center
    cropped_region = image.crop((crop_x1, crop_y1, crop_x2, crop_y2))

    # Resize the cropped region back to the original dimensions
    zoomed_image = cropped_region.resize((width, height))

    return zoomed_image

def add_background(background_path, transparent_image_path, output_path, zoom, zoom_factor=0.8, revert_zoom_order=False):
    try:
        # Open the background image
        background = Image.open(background_path).resize((128,128), Image.Resampling.LANCZOS)

        # Open the transparent image
        transparent_image = Image.open(transparent_image_path)


        if not revert_zoom_order:
            if zoom:
                transparent_image = zoom_center(transparent_image, zoom_factor)
            background = background.resize(transparent_image.size, Image.Resampling.LANCZOS)
        else:
            if zoom:
                background = zoom_center(background, zoom_factor)  
            transparent_image = transparent_image.resize(transparent_image.size, Image.Resampling.LANCZOS)
            


        # Composite the images
        result = Image.alpha_composite(background.convert('RGBA'), transparent_image)

        final_result = result.convert('RGB')
        # Save the result
                
        final_result.save(output_path, format='PNG')
    except Exception as ex:
        print(ex)
        return
    
# def find_keys_by_value(dictionary, value):
#     return [key for key, val in dictionary.items() if val == value]

In [ ]:
BACKGROUND_PATH = r"background.png"
ICONS_FOLDER = r"icons"
FINAL_FOLDER = r"icons_processed"

os.makedirs(FINAL_FOLDER, exist_ok=True)

names_dict = json.load(open("convert_names.txt", "r"))
reversed_dict = {}
 
for key, value in names_dict.items():
    if value not in reversed_dict:
        reversed_dict[value] = [key]
    else:
        reversed_dict[value].append(key)


for full_path in glob.glob(f"{ICONS_FOLDER}/*.png"):
    original_filename = Path(full_path).name
    new_filename_list = reversed_dict.get(original_filename, [None])
    for new_filename in new_filename_list:
        exception_list = ["Empty", "Coin", "Metal", "Wood"]
        if any(substring in original_filename for substring in exception_list):
            new_filename = original_filename
        elif not new_filename:
            new_filename = f"zzz{original_filename}"
        is_mega = new_filename.startswith("Mega_")
        output_path = os.path.join(FINAL_FOLDER, new_filename)
        if os.path.exists(output_path):
            continue
        if not is_mega:
            add_background(BACKGROUND_PATH, full_path, output_path, zoom=True, zoom_factor=0.95) 
        else:
            add_background(BACKGROUND_PATH, full_path, output_path, zoom=True, zoom_factor=0.8)

In [ ]:
BORDER_PATH = r"border.png"
ICONS_FOLDER = r"icons_processed"
FINAL_FOLDER = r"icons_extra"

os.makedirs(FINAL_FOLDER, exist_ok=True)

for full_path in glob.glob(f"{ICONS_FOLDER}/*.png"):
    original_filename = Path(full_path).name
    exception_list = ["Empty", "Coin", "Metal", "Wood"]
    if any(substring in original_filename for substring in exception_list):
        continue
    is_mega = original_filename.startswith("Mega_")
    output_path = os.path.join(FINAL_FOLDER, original_filename)
    if not is_mega:
        pass
        # add_background(full_path, BORDER_PATH, output_path, zoom=True, zoom_factor=0.95) 
    else:
        add_background(full_path, BORDER_PATH, output_path, zoom=True, zoom_factor=0.8, revert_zoom_order=True)